# 01. 소프트맥스 회귀의 비용 함수 구현하기

In [1]:
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

## 1. 로우레벨

In [3]:
z=torch.FloatTensor([1,2,3])

hypothesis=F.softmax(z,dim=0) # 1번째 차원에 대하여 소프트맥스 함수 적용
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
hypothesis.sum()

tensor(1.)

In [5]:
z=torch.rand(3,5,requires_grad=True)

In [8]:
hypothesis=F.softmax(z,dim=1) # 2번째 차원에 대하여 소프트맥스 함수 적용 == 가로줄 총합이 1
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [9]:
y=torch.randint(5,(3,)).long()
print(y)

tensor([0, 2, 1])


In [17]:
y_one_hot=torch.zeros_like(hypothesis)
y_one_hot.scatter_(1,y.unsqueeze(1), 1) # dim=1에 대해서 수행, y의 shape을 3x1로 바꾸고 1을 넣는다.

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [18]:
print(y.unsqueeze(1))

tensor([[0],
        [2],
        [1]])


In [16]:
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [19]:
cost=(y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


# 2. 하이-레벨
## 1. F.softmax() + torch.log() = F.log_softmax()

In [20]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [21]:
# High level
F.log_softmax(z,dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

## 2. F.log_softmax() + F.nll_loss() = F.cross_entropy()

In [24]:
# Low level
(y_one_hot * -torch.log(F.softmax(z,dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [25]:
(y_one_hot*-F.log_softmax(z,dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [27]:
# High level
F.nll_loss(F.log_softmax(z,dim=1),y)
# 원핫벡터 대신 실제값을 넣음

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [29]:
F.cross_entropy(z,y) # 내부에 소프트맥스 함수까지 포함하고 있으므로, 차원만 맞춰서 데이터 원형으로 넣으면 됨.

tensor(1.4689, grad_fn=<NllLossBackward0>)

# 02. 소프트맥스 회귀 구현하기

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [31]:
torch.manual_seed(1)

In [32]:
x_train=[[1,2,1,1],
        [2,1,3,2],
        [3,1,3,4],
        [4,1,5,5],
        [1,7,5,5],
        [1,2,5,6],
        [1,6,6,6],
        [1,7,7,7]]
y_train=[2,2,2,1,1,1,0,0]
x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)

## 1. 로우-레벨

In [33]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [34]:
y_one_hot=torch.zeros(8,3)
y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [35]:
W=torch.zeros((4,3),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

optimizer=optim.SGD([W,b],lr=0.1)

In [38]:
nb_epochs=1000
for epoch in range(nb_epochs+1):
    hypothesis=F.softmax(x_train.matmul(W)+b, dim=1)
    
    cost=(y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 0.985576
Epoch  100/1000 Cost: 0.759040
Epoch  200/1000 Cost: 0.688919
Epoch  300/1000 Cost: 0.642395
Epoch  400/1000 Cost: 0.603376
Epoch  500/1000 Cost: 0.567557
Epoch  600/1000 Cost: 0.533244
Epoch  700/1000 Cost: 0.499622
Epoch  800/1000 Cost: 0.466241
Epoch  900/1000 Cost: 0.432837
Epoch 1000/1000 Cost: 0.399290


## 2. 하이-레벨

In [40]:
W=torch.zeros((4,3),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

optimizer=optim.SGD([W,b], lr=0.1)

nb_epochs=1000
for epoch in range(nb_epochs +1):
    z=x_train.matmul(W)+b
    cost=F.cross_entropy(z,y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print('Epoch {:4d}/{} Cost:{:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost:1.098612
Epoch  100/1000 Cost:0.761050
Epoch  200/1000 Cost:0.689991
Epoch  300/1000 Cost:0.643229
Epoch  400/1000 Cost:0.604117
Epoch  500/1000 Cost:0.568256
Epoch  600/1000 Cost:0.533922
Epoch  700/1000 Cost:0.500291
Epoch  800/1000 Cost:0.466908
Epoch  900/1000 Cost:0.433507
Epoch 1000/1000 Cost:0.399962


## 3.  nn.Module로 구현하기

In [41]:
model=nn.Linear(4,3)

In [42]:
optimizer=optim.SGD(model.parameters(), lr=0.1)

nb_epochs=1000
for epoch in range(nb_epochs +1):
    prediction=model(x_train)
    
    cost=F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.658891
Epoch  200/1000 Cost: 0.573443
Epoch  300/1000 Cost: 0.518151
Epoch  400/1000 Cost: 0.473265
Epoch  500/1000 Cost: 0.433516
Epoch  600/1000 Cost: 0.396563
Epoch  700/1000 Cost: 0.360914
Epoch  800/1000 Cost: 0.325392
Epoch  900/1000 Cost: 0.289178
Epoch 1000/1000 Cost: 0.254148


## 4. 클래스로 구현하기

In [48]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(4,3)
        
    def forward(self, x):
        return self.linear(x)

In [49]:
model=SoftmaxClassifierModel()

In [50]:
optimizer=optim.SGD(model.parameters(), lr=0.1)

nb_epochs=1000
for epoch in range(nb_epochs+1):
    prediction=model(x_train)
    
    cost=F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.366217
Epoch  100/1000 Cost: 0.722726
Epoch  200/1000 Cost: 0.637564
Epoch  300/1000 Cost: 0.578576
Epoch  400/1000 Cost: 0.527363
Epoch  500/1000 Cost: 0.479315
Epoch  600/1000 Cost: 0.432700
Epoch  700/1000 Cost: 0.386693
Epoch  800/1000 Cost: 0.340930
Epoch  900/1000 Cost: 0.295757
Epoch 1000/1000 Cost: 0.255350
